In [1]:
import ciw
import scipy.stats
import pandas as pd
import matplotlib.pyplot as plt
import dask

In [2]:
def get_data_from_simulation(Q):
    recs = Q.get_all_records()
    data = pd.DataFrame(recs)
    data = pd.DataFrame(recs)
    data = data.sort_values('service_end_date')
    data['exit_order'] = data['id_number'].rank()
    data = data[['id_number', 'arrival_date', 'service_time', 'exit_order']]
    return data

In [3]:
def get_kendal_tau(Q, original_data):
    new_data = get_data_from_simulation(Q)
    tau = scipy.stats.kendalltau(original_data['exit_order'], new_data['exit_order'], variant='c')
    return tau.statistic

In [4]:
def get_average_tau(Qs, original_data):
    taus = [get_kendal_tau(Q, original_data) for Q in Qs]
    return sum(taus) / len(taus)

# Set up 'base' system, that is the system we wish to fit to

In [5]:
def SIRO_proportional(individuals):
    n = len(individuals)
    denominator = (n * (n + 1)) / 2
    probs = [(n - i) / denominator for i in range(n)]
    probs.reverse()
    return ciw.random_choice(individuals, probs=probs)

In [6]:
N = ciw.create_network(
    arrival_distributions=[ciw.dists.Exponential(9.5)],
    service_distributions=[ciw.dists.Deterministic(1/10)],
    number_of_servers=[1],
    service_disciplines=[SIRO_proportional],
)

In [7]:
ciw.seed(0)
Q = ciw.Simulation(N)
Q.simulate_until_max_time(100)

In [8]:
recs = Q.get_all_records()

In [9]:
data = pd.DataFrame(recs)
data = data.sort_values('arrival_date')
data['exit_order'] = data['exit_date'].rank()
data = data[['id_number', 'arrival_date', 'service_time', 'exit_order']]

In [10]:
data

,id_number,arrival_date,service_time,exit_order
0,1,0.195853,0.1,1.0
1,2,0.345183,0.1,2.0
3,3,0.402626,0.1,4.0
2,4,0.434168,0.1,3.0
4,5,0.509531,0.1,5.0
...,...,...,...,...
936,938,99.459491,0.1,937.0
937,939,99.555195,0.1,938.0
939,940,99.612280,0.1,940.0
938,941,99.643051,0.1,939.0


# Alternate base system, for comparison

In [11]:
N = ciw.create_network(
    arrival_distributions={
        'C0': [ciw.dists.Exponential(10)],
        'C1': [None]},
    service_distributions={
        'C0': [ciw.dists.Deterministic(1/10.2)],
        'C1': [ciw.dists.Deterministic(1/10.2)]},
    number_of_servers=[1],
    class_change_time_distributions={
        'C0': {'C1': ciw.dists.Exponential(3)},
        'C1': {'C0': ciw.dists.Exponential(8)}},
    priority_classes={
        'C0': 1,
        'C1': 0}
)

In [12]:
ciw.seed(0)
Q = ciw.Simulation(N)
Q.simulate_until_max_time(250)

In [13]:
recs = Q.get_all_records()

In [14]:
data = pd.DataFrame(recs)
data = data.sort_values('arrival_date')
data['exit_order'] = data['exit_date'].rank()
data = data[['id_number', 'arrival_date', 'service_time', 'exit_order']]

In [15]:
data

,id_number,arrival_date,service_time,exit_order
0,1,0.186061,0.098039,1.0
1,2,0.240632,0.098039,2.0
2,3,0.312228,0.098039,3.0
3,4,0.465382,0.098039,4.0
4,5,0.530122,0.098039,5.0
...,...,...,...,...
2496,2544,246.945590,0.098039,2497.0
2492,2547,247.134485,0.098039,2493.0
2506,2559,248.416437,0.098039,2507.0
2499,2560,248.586394,0.098039,2500.0


In [16]:
1 - scipy.stats.kendalltau(data['id_number'], data['exit_order']).statistic

0.027690592750560183

# Set up

In [17]:
def create_candidate_network(theta_01, theta_10, original_data):
    inter_arrivals = list(original_data['arrival_date'].diff().fillna(0))
    dist_01 = None
    dist_10 = None
    if theta_01 > 0.0:
        dist_01 = ciw.dists.Exponential(rate=theta_01)
    if theta_10 > 0.0:
        dist_10 = ciw.dists.Exponential(rate=theta_10)
    N = ciw.create_network(
        arrival_distributions={
            'C0': [ciw.dists.Sequential(inter_arrivals)],
            'C1': [None]},
        service_distributions={
            'C0': [ciw.dists.Deterministic(1/5)],
            'C1': [ciw.dists.Deterministic(1/5)]},
        number_of_servers=[1],
        class_change_time_distributions={
            'C0': {'C1': dist_01},
            'C1': {'C0': dist_10}},
        priority_classes={
            'C0': 1,
            'C1': 0}
    )
    return N

In [18]:
def run_one_trial(N, n_custs, seed):
    ciw.seed(seed)
    Q = ciw.Simulation(N)
    Q.simulate_until_max_customers(n_custs)
    return Q

In [19]:
def run_trials(theta_01, theta_10, original_data, n_trials):
    n_custs = len(original_data)
    Qs = []
    N = create_candidate_network(theta_01, theta_10, original_data)
    
    tasks = [dask.delayed(run_one_trial)(N=N, n_custs=n_custs, seed=seed) for seed in range(n_trials)]
    Qs = dask.compute(*tasks, num_workers=8)
    return Qs

In [20]:
def evaluate(theta, original_data=data):
    theta_01 = theta[0]
    theta_10 = theta[1]
    Qs = run_trials(theta_01=theta_10, theta_10=theta_10, original_data=original_data, n_trials=16)
    average_tau = get_average_tau(Qs, original_data)
    return 1 - average_tau

In [21]:
evaluate(theta=[0, 0], original_data=data)

0.02769059275055996

# Try to fit

In [22]:
import nevergrad as ng

In [ ]:
# optimization on x as an array of shape (2,)
instrum = ng.p.Instrumentation(
    ng.p.Array(shape=(2,), lower=0),
)
optimizer = ng.optimizers.NGOpt(parametrization=instrum, budget=50, num_workers=1)
optimizer.register_callback("tell", ng.callbacks.ProgressBar())
recommendation = optimizer.minimize(evaluate)  # best value
print(recommendation.value)

In [ ]:
evaluate(recommendation.value[0][0])

ERROR! Session/line number was not unique in database. History logging moved to new session 3800


Custom TB Handler failed, unregistering


In [23]:
evaluate([2, 5])

ERROR! Session/line number was not unique in database. History logging moved to new session 3801



KeyboardInterrupt



In [ ]:
plt.plot(
    [r[0] for r in optimizer.optim_curve],
    [r[1] for r in optimizer.optim_curve]
)

In [ ]:
plt.plot([r.mean for r in list(optimizer.archive.values())])